# One-Dimension Line Search 一维线搜索

## 问题描述

Consider a rectangular wing of span $b$ and chord $c$. Its planform area is thus $ S=b c $ and its aspect ratio is $ A = b^{2} / S $. The drag of this wing can be approximated as,
$$
C_{D}=k C_{f} \frac{S_{w e t}}{S}+\frac{C_{L}^{2}}{\pi A e}
$$
The first term corresponds to the parasite drag. $ C_{f} $ is the skin friction coefficient, which for a fully turbulent boundary layer can be approximated as,
$$
C_{f}=\frac{0.074}{R e^{0.2}}
$$
Here, the Reynolds number $ (R e=\rho V c / \mu) $ is based on the wing chord. $ k $ is the form factor, which accounts for the effects of pressure drag.

The second term in Equation (1) is the induced drag, where $ e $ is the Oswald efficiency factor. The lift coefficient $ C_{L} $ and the wing planform area $ S $ are to be kept constant. The values for all the constants are listed in Table 1. 
1. Write the total drag coefficient as a function of $ A $.
2. Minimize $ C_{D} $ with respect to $ A $ using:   
    (a) The golden section method  
    (b) A line search method that satisfies sufficient decrease. (Bonus: A line search that satisfies the strong Wolfe conditions.)  
    Converge the solutions to 6 significant digits.  
3. Discuss the relative performance of these two methods. Try different starting points/intervals and compare convergence rates, number of iterations and any other metrics you find suitable.

\begin{array}{lrll}
\text { Quantity } & \text { Value } & \text { Units } & \text { Description } \\
\hline \rho & 1.23 & \mathrm{~kg} / \mathrm{m}^{3} & \text { density of air } \\
\mu & 17.8 \times 10^{-6} & \mathrm{~kg} /(\mathrm{m} \mathrm{sec}) & \text { viscosity of air } \\
V & 35 & \mathrm{~m} / \mathrm{s} & \text { airspeed } \\
S & 11.8 & \mathrm{~m}^{2} & \text { planform area } \\
S_{\text {wet }} & 2.05 S & \mathrm{~m}^{2} & \text { wing wetted area } \\
k & 1.2 & & \text { form factor } \\
C_{L} & 0.3 & & \text { lift coefficient } \\
e & 0.96 & & \text { Oswald efficiency factor } \\
\hline
\end{array}


In [1]:
import numpy as np

def Cal_Re(c, rho=1.23, V=35.0, mu=17.8e-6):
    """Calculate Reynolds number."""
    return rho*V*c/mu

def Cal_Cf(Re):
    """Calculate the skin friction coefficient"""
    return 0.074/(Re**0.2)

def Cal_Cd(A, k=1.2, S=11.8, S_ratio=2.05, Cl=0.3, e=0.96):
    """Calculate the total drag coefficient as the function of A"""
    b = np.sqrt(A*S)
    c = S/b
    Re = Cal_Re(c)
    Cf = Cal_Cf(Re)
    return k*Cf*S_ratio+Cl**2/(np.pi*A*e)

## The Golden Section Method 黄金切割法

In [4]:
class GoldenSectionMethod():
    """Golden Section Method"""
    def __init__(self, function, x_min=1.0, x_max=100.0, tau=0.618, error = 1e-6):
        self.x_max = x_max
        self.x_min = x_min
        self.function = function
        self.tau = tau
        self.error = error
        self.recordmn = []
        self.recordmin = []
        self.recordfx = []

    def Search(self, iter_max=2000):
        # init step
        b = self.x_max
        a = self.x_min
        n = max( a + (b-a)*self.tau, b - (b-a)*self.tau )
        m = min( a + (b-a)*self.tau, b - (b-a)*self.tau )
        #self.recordmn.append([m, n])
        # search
        iteration = 1
        min_index_old = 100000
        while(iteration<=iter_max):
            if( self.function(m) < self.function(n)):
                min_index = m
                b = n
            elif( self.function(m) > self.function(n)):
                min_index = n 
                a = m
            else:
                min_index = (m+n)/2.0
                a = m
                b = n
            #if(self.function(min_index)>self.function((a+b)/2.0)):
            min_index = (a+b)/2.0
            self.recordmin.append([iteration, self.function(min_index)])
            self.recordfx.append([min_index, self.function(min_index)])

            n = max( a + (b-a)*self.tau, b - (b-a)*self.tau )
            m = min( a + (b-a)*self.tau, b - (b-a)*self.tau )
            self.recordmn.append([a, b])

            if(abs(min_index - min_index_old)<=self.error):
                #print("The accuracy has reached "+str(self.error))
                print('Iteration = '+str(iteration))
                break

            min_index_old = min_index
            iteration += 1

        return min_index, self.function(min_index)

    def output(self, flag_fx=1,flag_converage = 1, flag_mn = 1):
        if(flag_fx == 1):
            with open("./outdata/1_One-Dimension_Line_Search/GoldenSection_fx.dat", 'w') as f:
                f.write("Variables = x, f(x) \n")
                f.write("zone \n")
                x = np.linspace(self.x_min,self.x_max,1000)
                for i in x:
                    f.write(str(i)+' '+str(self.function(i))+ "\n")
                f.write("zone \n")
                for j in self.recordfx:
                    f.write(str(j[0])+' '+str(j[1])+ "\n")

        if(flag_converage == 1):
            with open("./outdata/1_One-Dimension_Line_Search/GoldenSection_converage.dat", 'w') as f:
                f.write("Variables = x, f(x) \n")
                f.write("zone \n")
                for j in range(len(self.recordmin)):
                    f.write(str(j)+' '+str(self.recordmin[j][1])+ "\n")

        if(flag_mn == 1):
            with open("./outdata/1_One-Dimension_Line_Search/GoldenSection_x1x2.dat", 'w') as f:
                f.write("Variables = x, f(x) \n")
                i = 0.01
                for j in self.recordmn:
                    f.write("zone \n")
                    f.write(str(j[0])+' '+str(i)+ "\n")
                    f.write(str(j[1])+' '+str(i)+ "\n")
                    i = i+0.0001
                    

Golden = GoldenSectionMethod(function=Cal_Cd)
A_min, Cd_min = Golden.Search()
Golden.output()

for i in range(10):
    Golden = GoldenSectionMethod(function=Cal_Cd, x_min=1.0, x_max=float(10+i*10))
    A_min, Cd_min = Golden.Search()
    print(A_min,Cd_min)


Iteration = 36
Iteration = 31
9.999998507703083 0.012452367964004061
Iteration = 33
19.999998796784872 0.011639856490705125
Iteration = 34
28.394248943020834 0.011560688987166723
Iteration = 33
28.39424728323062 0.011560688987166721
Iteration = 34
28.3942482150752 0.011560688987166721
Iteration = 35
28.394248650832544 0.011560688987166723
Iteration = 36
28.39424726269298 0.011560688987166721
Iteration = 33
28.39424688677658 0.011560688987166723
Iteration = 35
28.394248642062152 0.01156068898716672
Iteration = 36
28.394248367400227 0.011560688987166718


## A line search that satisfies the strong Wolfe conditions.

对于一维搜索问题，定义：
$$
\phi (A_{k} + \alpha) = C_{d} = k C_{f} \frac{S_{w e t}}{S}+\frac{C_{L}^{2}}{\pi (A_{k} + \alpha) e}
$$
其中，
$$
C_{f} = \frac{0.074}{R e^{0.2}} \\
R e = \frac{\rho V c }{\mu} = \frac{\rho V \sqrt{S} }{\mu} \frac{1}{\sqrt{A_{k} + \alpha}}
$$
则函数$\phi$的一阶导数为：
$$
\phi^{'} (\alpha) = \frac{\mathrm{d} C_{d}}{\mathrm{d} \alpha} 
= k \frac{S_{w e t}}{S} \frac{\mathrm{d} C_{f}}{\mathrm{d} \alpha} - \frac{C_{L}^{2}}{\pi (A_{k} + \alpha)^{2} e}
$$

其中，
$$
\frac{\mathrm{d} C_{f}}{\mathrm{d} \alpha} = - \frac{0.2 \times 0.074}{R e^{1.2}} \frac{\mathrm{d} Re}{\mathrm{d} \alpha}
= \frac{0.2 \times 0.074}{R e^{1.2}} \frac{ \rho V \sqrt{S}}{\mu} (0.5*\frac{1}{(A_{k} + \alpha)^{1.5}})
$$


In [11]:
def grad_Cd(A, k=1.2, S=11.8, S_ratio=2.05, Cl=0.3, e=0.96, rho=1.23, V=35.0, mu=17.8e-6):
    """Calculate the gradient of Cd as the function of A"""
    b = np.sqrt(A*S)
    c = S/b
    Re = Cal_Re(c)
    phi1 = k*S_ratio*(0.2*0.074)/(Re**1.2)*rho*V*np.sqrt(S)/mu*0.5*(A**-1.5)
    phi2 = Cl**2/(np.pi * A**2 * e)

    return phi1 - phi2

class WolfeConditionsSearch():
    """A line search that satisfies the strong Wolfe conditions."""
    def __init__(self, xfunction, p, x_init=1.0, x_min=1.0, x_max=200.0, mu1=1e-4, mu2=0.9):
        self.xfunction = xfunction
        self.x_max = x_max
        self.x_min = x_min
        self.mu1 = mu1
        self.mu2 = mu2
        self.x_init = x_init
        self.p = p
        if( x_init<x_min or x_init>x_max ):
            print("the start point is beyond the boundry")
            return None
    
    def function(self, a):
        return self.xfunction(self.x_init + a * self.p)

    def gfunction(self, a, h=0.0001):
        return (self.function(a+h)-self.function(a-h))/(2*h)

    def lineSearch(self, a_step = 1 , iter_max=2000):
        # Search the space of a
        a_best = 0
        a0 = 0
        a = a0 + a_step
        find = False
        iteration = 1    
        while(iteration<iter_max):
            if(self.x_init + a * self.p < self.x_min ):
                a_min = a - a_step
                a_max = self.x_init - self.x_min
                break
            elif(self.x_init + a * self.p > self.x_max):
                a_min = a - a_step
                a_max = self.x_max - self.x_init
                break
            if(self.function(a) > self.function(a0) + self.mu1*a*self.gfunction(a0)):
                a_min = a - a_step
                a_max = a 
                break
            if(abs(self.gfunction(a)) < abs(self.mu2*self.gfunction(a0)) ):
                a_best = a
                find = True 
                break
            if(self.gfunction(a)*self.gfunction(a0)<0):
                a_min = a - a_step
                a_max = a 
                break
            a += a_step
            iteration += 1
        if(iteration>=iter_max):
            print("There is no minimum point")
            return 404

        if(find):
            return self.x_init + a_best * self.p
            
        # reduce the space of a
        iteration = 1
        while(iteration<iter_max):  
            a = (a_min + a_max) / 2.0
            if(self.x_init + a * self.p < self.x_min or self.x_init + a * self.p > self.x_max):
                print("the step of a is too large")
                return 404
            if(self.function(a) > self.function(a0)+self.mu1*a*self.gfunction(a0)):
                a_max = a 
                continue
            if(abs(self.gfunction(a)) < abs(self.mu2*self.gfunction(a0))):
                a_best = a
                find = True 
                break
            if(self.gfunction(a)*(a_max - a_min)>0):
                a_max = a 
            elif(self.gfunction(a)*(a_max - a_min)<0):
                a_min = a
            iteration += 1
        
        if(find):
            return self.x_init + a_best * self.p
        else:
            print("The iteration has reach the maximum number")
            return 404


with open("./outdata/1_One-Dimension_Line_Search/WolfeCondition_converage.dat", 'w') as f:
    f.write("Variables = iteration, f(x) \n")
    for i in range(10):
        f.write("zone \n")
        init_x = 1.0 + i*10.0
        iteration = 1
        error = 10000
        f.write(str(iteration)+' '+str(Cal_Cd(init_x))+ "\n")
        while( error > 1e-6):
            p =  - grad_Cd(init_x)/abs(grad_Cd(init_x))
            wolfe = WolfeConditionsSearch(Cal_Cd, p, x_init=init_x)
            a = wolfe.lineSearch(a_step=10, iter_max=1000)
            if(a == 404):
                break
            #error = abs(Cal_Cd(init_x+a)-Cal_Cd(init_x))
            error = abs(a - init_x)
            init_x = a 
            iteration += 1
            f.write(str(iteration)+' '+str(Cal_Cd(a))+ "\n")
        print(iteration)
        print(init_x, Cal_Cd(init_x))   

17
28.39424765110016 0.01156068898716672
16
28.39424765110016 0.01156068898716672
15
28.39424765110016 0.01156068898716672
14
28.39424765110016 0.01156068898716672
15
28.39424765110016 0.01156068898716672
16
28.39424765110016 0.01156068898716672
16
28.39424765110016 0.01156068898716672
16
28.39424765110016 0.01156068898716672
16
28.39424765110016 0.01156068898716672
15
28.39424765110016 0.01156068898716672
